---

## Function Definitions

In this section, we define some of the necessary functions required for our analysis.

1. **`country_region_mapping(df)`**: This function takes a dataframe and maps each country code in the 'DHSID' column to its corresponding region. It first creates a dictionary mapping from country codes to regions, then extracts the country codes from the 'DHSID' column, corrects any mislabeled country codes, and finally maps the regions and countries to a new column 'target_region'.

2. **`split_data_country_wise(df)`**: This function splits the data into training, development, and test sets for each country, by dropping the label columns, and then for each unique country, splits the data into training *(80%)*, development *(10%)*, and test *(10%)* sets. It then concatenates all the training sets, all the development sets, and all the test sets to form the final `X_train`, `X_dev`, `X_test`, `y_train`, `y_dev`, and `y_test`. *This is done to ensure that each set has representation from all the countries.*


3. **`one_hot_encode_fit(X_train)`**: This function one-hot encodes the categorical columns of the training data. It takes the training data, identifies the categorical columns, and then applies one-hot encoding to these columns using the `ColumnTransformer` and `OneHotEncoder` classes from `sklearn`. It returns the encoded training data and the transformer.

4. **`one_hot_encode_transform(X, transformer)`**: This function applies one-hot encoding to a dataframe using a previously fitted transformer. It takes a dataframe and a transformer, applies the transformer to the dataframe, and returns the encoded dataframe.

5. **`obj_columns(df)`**: This function takes a dataframe and returns a list of the column names of the columns with object datatype.

6. **`mcrmse(y_true, y_pred)`**: This function computes the mean column-wise root mean square error between the true and predicted labels. It takes the true labels and the predicted labels, computes the square error between them, takes the mean of the square errors for each label, takes the square root of these means, and then takes the mean of all the square roots.

7. **`make_predictions(input_df)`**: This function preprocesses the input dataframe and uses an ensemble of trained models to make predictions. It takes the input dataframe, ensures it has been preprocessed, loads the trained models, scores, and weights, makes predictions using each model in the ensemble, computes the ensemble predictions, and then converts the predictions to a dataframe with appropriate column names.

---



In [ ]:
import numpy as np
import pandas as pd
import joblib
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


def country_region_mapping(df):
    # Country code to region mapping
    country_to_region = {}
    country_to_region.update({code: "East Asia & Pacific" for code in {"BD", "KH", "ID", "LA", "MM", "NP", "PH", "TL", "VN", "GU"}})
    country_to_region.update({code: "Central Asia" for code in {"TJ", "UZ", "KG", "KZ"}})
    country_to_region.update({code: "Europe & Central Asia" for code in {"AL", "AM", "AZ", "GE", "MD", "MK", "RS", "TR", "UA"}})
    country_to_region.update({code: "Sub-Saharan Africa" for code in {"AO", "BJ", "BF", "BU", "CM", "CF", "CI", "CD", "ET", "GA", "GH", "GN", "GY", "KE", "KM", "LS", "LB", "MD", "MW", "ML", "MZ", "NG", "NM", "RW", "SN", "SL", "SZ", "TD", "TG", "ZA", "TZ", "UG", "ZM", "ZW"}})
    country_to_region.update({code: "North Africa & Middle East" for code in {"EG", "JO", "YE", "MA", "LB", "MB"}})
    country_to_region.update({code: "South Asia" for code in {"AF", "BD", "IN", "PK", "NP", "IA"}})
    country_to_region.update({code: "Latin America & Caribbean" for code in {"BO", "CO", "DR", "HT", "HN", "MX", "PE", "NI"}})

    # Extract country codes from DHSID
    df['Country_Code'] = df['DHSID'].str.extract(r'([A-Za-z]+)')[0]
    
    # Correct any mislabeled country codes
    df.loc[df["Country_Code"] == "DHS", "Country_Code"] = "BD"

    # Map regions and countries
    df['target_region'] = df['Country_Code'].map(country_to_region)

    return df

def split_data_country_wise(df):
    X = df.drop(columns=labels)
    y = df[labels]
    # Unique country codes
    countries = X['Country_Code'].unique()

    # Initialize empty lists to store the split data
    X_train_list, X_dev_list, X_test_list = [], [], []
    y_train_list, y_dev_list, y_test_list = [], [], []

    # Split the data for each country
    for country in countries:
        X_country = X[df['Country_Code'] == country]
        y_country = y[df['Country_Code'] == country]
        
        X_train_country, X_temp, y_train_country, y_temp = train_test_split(X_country, y_country, test_size=0.2, random_state=1)
        X_dev_country, X_test_country, y_dev_country, y_test_country = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

        X_train_list.append(X_train_country)
        X_dev_list.append(X_dev_country)
        X_test_list.append(X_test_country)

        y_train_list.append(y_train_country)
        y_dev_list.append(y_dev_country)
        y_test_list.append(y_test_country)

    # Concatenate the splits
    X_train = pd.concat(X_train_list, ignore_index=True)
    X_dev = pd.concat(X_dev_list, ignore_index=True)
    X_test = pd.concat(X_test_list, ignore_index=True)

    y_train = pd.concat(y_train_list, ignore_index=True)
    y_dev = pd.concat(y_dev_list, ignore_index=True)
    y_test = pd.concat(y_test_list, ignore_index=True)

    return X_train,X_dev,X_test,y_train,y_dev,y_test

def one_hot_encode_fit(X_train):
    # Specify the columns to be encoded
    categorical_columns = X_train.select_dtypes(include=['object']).columns.tolist()
    # Create transformer
    transformer = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)],
        remainder='passthrough')
    X_train_encoded = transformer.fit_transform(X_train)
    return X_train_encoded, transformer

def one_hot_encode_transform(X, transformer):
    X_encoded = transformer.transform(X)
    return X_encoded

def obj_columns(df):
    return(df.columns[df.dtypes == 'object'].tolist())

def mcrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean(np.square(y_true - y_pred), axis=0)))

def make_predictions(input_df):
    # Ensure the input dataframe has been preprocessed
    # Load the trained models, scores, and weights
    chains = joblib.load('chains.joblib')
    weights = joblib.load('weights.joblib')
    
    # Make predictions using each model in the ensemble
    predictions = [chain.predict(input_df) for chain in chains]
    
    # Compute the ensemble predictions
    final_predictions = np.zeros_like(predictions[0])
    for weight, prediction in zip(weights, predictions):
        final_predictions += weight * prediction

    # Convert predictions to a dataframe with appropriate column names
    final_predictions_df = pd.DataFrame(final_predictions, columns=labels)
        
    return final_predictions_df


---
## Managing Large Datasets
The initial challenge in our project was the management of multiple large datasets that exceeded memory limits. To address this, we implemented two key strategies to make data manipulation more efficient.

- We converted the dataset from CSV to Parquet format. This significantly improved memory efficiency and enabled faster data access and processing.

- We downcasted the variables to the lowest possible datatype without losing precision, reducing the memory footprint of the dataset and facilitating efficient data handling.
---
## Data Preprocessing
In this step, we load the `gee_features` and `training labels` file, drop irrelevant columns, and remove duplicate data points.

---

In [ ]:
df = pd.read_parquet("/kaggle/input/stanford-data/gee_features_10pct.parquet")
df.drop_duplicates(subset = 'DHSID',inplace=True)
df.drop(columns = ['key1','DHSCC','DATUM','CCFIPS','SOURCE','new_ind','ADM1FIPS','ADM1FIPSNA','ADM1NAME','ADM1SALBCO','ADM1SALBNA','ADM1SALNA','F21','F22','F23','ZONECO','ZONENA'], inplace = True)

In [ ]:
df.head()

In [ ]:
tl = pd.read_csv("/kaggle/input/stanford-data/training_label.csv")
tl.drop_duplicates(subset = 'DHSID',inplace=True)
tl.drop(columns=['URBAN_RURA','DHSCLUST','DHSYEAR', 'LATNUM', 'LONGNUM'],inplace = True)

labels = ['Mean_BMI', 'Median_BMI', 'Unmet_Need_Rate', 'Under5_Mortality_Rate','Skilled_Birth_Attendant_Rate', 'Stunted_Rate']

In [ ]:
tl.head()

## Merging Ground Truth and Features
To prepare the data for model training, we need to merge the ground truth (`training labels`) with the `gee_features` based on the common column 'DHSID'. This will ensure that each row in the dataset has both the features and the corresponding label.


In [ ]:
merged_df1 = pd.merge(df, tl, on='DHSID', how='right')
merged_df1 = merged_df1.dropna(subset=labels)
merged_df1 = merged_df1.dropna(subset = merged_df1[['key3']].columns.to_list())
merged_df1.reset_index(inplace=True,drop=True)

# del df,tl
# gc.collect()

## Additional Data

We have integrated additional data from elite global source, such as the *World Health Organisation*.

The data was queried using the following methods:

- **World Bank Data**: The data was queried using the `wbdata API` (Python module) by feeding in the country and the year, then the retrieved data was transformed and stored in a dataframe, with `DHSID` used as the key to merge with initial data.
-The data was retireved only for unique `Country_Code` - `DHSYEAR` pair, and then appended to the dataframe for all the countries

The workflow for retrieving this data has been designed to save progress periodically(written to a `pickle` file), allowing the process to be restarted from the last saved checkpoint if data retrieval is stopped or fails.

More information about the workflow and the data can be found here: [GitHub Link](your_github_link_here)

In [ ]:
df_worldbank = pd.read_csv("/kaggle/input/stanford-data/WorldBank_Features.csv")
df_worldbank.drop(columns = ['DHSYEAR','Country_Code','SpatialDim','TimeDim'],inplace = True)
merged_df = pd.merge(merged_df1, df_worldbank, on='DHSID', how='left')

### Feature Selection
Selecting the *optimum number of features*, as well as the *most relevant features*, is crucial for building an efficient model. For this purpose, we train a boosting model (**XGBoost**) using a *multi-target regression wrapper* on all the merged features. We then compute the **feature importance** for each feature. Evaluation is done using *k-fold cross-validation*.



In [ ]:
%%time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

fea_df = merged_df.drop(columns = labels)
target_labels = merged_df[labels]

# Encode categorical variables using label encoding
label_encoders = {}
for column in fea_df.select_dtypes(include='object').columns:
    label_encoders[column] = LabelEncoder()
    fea_df[column] = label_encoders[column].fit_transform(fea_df[column])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(fea_df, target_labels, test_size=0.2, random_state=42)


# Train the XGBoost model using the MultiOutputRegressor wrapper
model = MultiOutputRegressor(xgb.XGBRegressor(objective ='reg:squarederror'))
model.fit(X_train, y_train)

# Compute the feature importances
feature_importances = np.zeros(X_train.shape[1])
for estimator in model.estimators_:
    feature_importances += estimator.feature_importances_

# Normalize the feature importances
feature_importances /= np.sum(feature_importances)

# Create a DataFrame to hold the feature importances
importances_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by the importances
importances_df = importances_df.sort_values(by='Importance', ascending=False)
importances_df.to_csv("Feature_Importance.csv",index = False)
# del fea_df,target_labels,df,X_train, X_test, y_train, y_test
# gc.collect()

## Optimum Number of Features

The optimum number of features is then selected based on the graph of **MCRMSE** (Mean Column-wise Root Mean Square Error) score *(k-fold mean)* vs the *number of features*. The code below plots the graph for it. Based on the graph, we can observe that there is an slight elbow at the **number of features = 350**, so we choose it as the **optimum number of features**.


In [ ]:
import matplotlib.pyplot as plt
import xgboost as xgb
from tqdm.notebook import tqdm
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor

sorted_features = importances_df['Feature'].values

X_train, X_dev, X_test, y_train, y_dev, y_test = split_data_country_wise(country_region_mapping(merged_df).drop(columns=['DHSID']))

# Define the range of number of features to use
# In our actual code, we tried running till no. of features = 1000
num_features_range = range(10, 501, 20)

# Initialize a list to hold the validation scores
validation_scores = []

# For each number of features in the range
for num_features in tqdm(num_features_range):
    # Select the top num_features most important features
    selected_features = sorted_features[:num_features]
    X_train_selected = X_train[selected_features]
    X_dev_selected = X_dev[selected_features]
    
    X_train_selected, encoder = one_hot_encode_fit(X_train_selected)
    X_dev_selected = one_hot_encode_transform(X_dev_selected,encoder)
    
    # Train the model
    model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror'))
    model.fit(X_train_selected, y_train)
    
    y_pred = model.predict(X_dev_selected)
    
    # In the actual code, K fold Validation(folds = 10) was used so that we get a robust idea of evaluation
    validation_score = mcrmse(y_dev, y_pred)
    validation_scores.append(validation_score)

# Plot the validation scores against the number of features
# Plot the validation scores against the number of features
plt.plot(num_features_range, validation_scores)
plt.xlabel('Number of Features')
plt.ylabel('Validation Score')
plt.title('Validation Score vs Number of Features')
plt.grid()
plt.show()


#del X_train, X_dev, X_test, y_train, y_dev, y_test
#gc.collect()

In [ ]:
X_train, X_dev, X_test, y_train, y_dev, y_test = split_data_country_wise(merged_df.drop(columns=['DHSID']))

num_features = 350
# In actual run we used 350 features based on the plot, more info about it can be found in the report
sorted_features = importances_df['Feature'].values
selected_features = sorted_features[:num_features]

X_train = X_train[selected_features]
X_dev = X_dev[selected_features]
X_test = X_test[selected_features]

In [ ]:
X_train, encoder = one_hot_encode_fit(X_train)
X_dev = one_hot_encode_transform(X_dev, encoder)
X_test = one_hot_encode_transform(X_test, encoder)

In [ ]:
X_train = pd.DataFrame(X_train)
X_dev = pd.DataFrame(X_dev)
X_test = pd.DataFrame(X_test)

X_train_cols = X_train.columns.to_list()

## Imputation of Missing Values

In this section, we handle missing values in our dataset. Missing data is a common occurrence in real-world data and needs to be addressed before training a model. We use the K-nearest neighbors (KNN) algorithm to impute missing values in the numerical columns of our dataset.

1. Separate out the numerical columns from the dataset.
2. Apply KNN imputation on the numerical columns.
3. Update the original dataset with the imputed numerical values.
4. Impute the remaining missing values in the dataset using the median of each column.

The KNN imputer uses the mean of the 'k' most similar instances (or neighbors) to impute missing values. We use **'k=5'** for our imputer(this was decided after comprehensive testing with various k values). If there are still any missing values left after the KNN imputation, we fill them with the median of the corresponding column.



In [ ]:
from sklearn.impute import KNNImputer

# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=5)

# 1. Separate out the numerical columns
numerical_cols = X_train.select_dtypes(exclude=['object']).columns

X_train_numerical = X_train[numerical_cols].copy()
X_dev_numerical = X_dev[numerical_cols].copy()
X_test_numerical = X_test[numerical_cols].copy()

# 2. Apply KNN imputation on numerical columns
X_train_numerical_imputed = imputer.fit_transform(X_train_numerical)
X_dev_numerical_imputed = imputer.transform(X_dev_numerical)
X_test_numerical_imputed = imputer.transform(X_test_numerical)

# Convert imputed data back to dataframe
X_train_numerical_df = pd.DataFrame(X_train_numerical_imputed, columns=numerical_cols, index=X_train.index)
X_dev_numerical_df = pd.DataFrame(X_dev_numerical_imputed, columns=numerical_cols, index=X_dev.index)
X_test_numerical_df = pd.DataFrame(X_test_numerical_imputed, columns=numerical_cols, index=X_test.index)

# 3. Update the original dataset with the imputed numerical values
X_train.update(X_train_numerical_df)
X_dev.update(X_dev_numerical_df)
X_test.update(X_test_numerical_df)

#imputating the remaining values with the median of the data
X_train = X_train.fillna(X_train.median())
X_dev = X_dev.fillna(X_train.median())
X_test = X_test.fillna(X_train.median())

In [ ]:
X_train.head()

## Optimum Chaining Sequence using Greedy Algorithm

8. **`evaluate_order(model, order, X_train, y_train, X_dev, y_dev)`**: This function evaluates the mean column-wise root mean square error (MCRMSE) of a model with a given order of the labels. It takes the model, the current order of the labels, the training and development sets, completes the ordering of the labels, fits a `RegressorChain` with the given order on the training data, predicts the development data, and computes the MCRMSE of the predictions.

9. **`full_greedy_search(model, X_train, y_train, X_dev, y_dev)`**: This function finds the best order of the labels for a given model using a greedy algorithm. It takes the model and the training and development sets, initializes the current order as empty and the set of all targets as all the labels, and iteratively adds the label that results in the lowest MCRMSE to the current order, until all labels have been added to the order. It returns the best order found and the history of orders and scores.

These two functions are used in finding the best order for the regression chain. They use a greedy algorithm to find the next label to be added to the order.

In [ ]:
%%time
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.multioutput import RegressorChain

def evaluate_order(model, order, X_train, y_train, X_dev, y_dev):
    # Complete the ordering
    full_order = order + [i for i in range(y_train.shape[1]) if i not in order]

    chain = RegressorChain(model, order=full_order)
    chain.fit(X_train, y_train)
    y_pred = chain.predict(X_dev)
    score = mcrmse(y_dev, y_pred)

    return score

def full_greedy_search(model, X_train, y_train, X_dev, y_dev):
    all_targets = list(range(y_train.shape[1]))
    current_order = []
    history = []
    
    while all_targets:
        best_score = float('inf')
        best_target = None

        for t in all_targets:
            temp_order = current_order + [t]
            score = evaluate_order(model, temp_order, X_train, y_train, X_dev, y_dev)
            
            # Capture history for insights on progression
            history.append((temp_order, score))
            
            if score < best_score:
                best_score = score
                best_target = t

        print(f"Added target {best_target} to the order. Current best score: {best_score}")
        
        current_order.append(best_target)
        all_targets.remove(best_target)

    return current_order, history

lgb_model = LGBMRegressor()
xgb_model = XGBRegressor()
cat_model = CatBoostRegressor(verbose=0)

print(75*"-")
xgb_best_order, search_history = full_greedy_search(xgb_model, X_train, y_train, X_dev, y_dev)
print(f"Best order found for xgb: {xgb_best_order}")
print(75*"-")
lgb_best_order, search_history = full_greedy_search(lgb_model, X_train, y_train, X_dev, y_dev)
print(f"Best order found for lgbm: {lgb_best_order}")
print(75*"-")
cat_best_order, search_history = full_greedy_search(cat_model, X_train, y_train, X_dev, y_dev)
print(f"Best order found for cat: {cat_best_order}")
print(75*"-")

## Hyperparameter Optimization using Optuna

In this section, we use Optuna for hyperparameter tuning. Optuna is an open-source hyperparameter optimization framework in Python. It is designed to optimize machine learning model's hyperparameters with a user-friendly interface, which is key to the efficient development of machine learning models.

The `objective` function defined below is what Optuna will optimize. It takes an Optuna `trial` object and:

1. Suggests a model type (XGBoost, LightGBM, or CatBoost) and the corresponding hyperparameters for the model.
2. Fits a `RegressorChain` with the suggested model and hyperparameters and the best order of the labels found in the previous section on the training data.
3. Predicts the development data and computes the MCRMSE of the predictions.

Optuna will then try to find the hyperparameters that minimize the MCRMSE.

We use a `MedianPruner` to stop the optimization of trials that do not appear promising, and run the optimization for 50 trials. In the actual run, we conducted the optimization with 200 trials.

After the optimization, we print the best parameters and score for each model type.



In [ ]:
%%time
import optuna
def objective(trial):
    try:
        # 1. Model selection
        model_type = trial.suggest_categorical("model_type", ["xgb", "lgb", "cat"])

        # 2. Hyperparameter definitions based on the model
        if model_type == "xgb":
            model = XGBRegressor(
                random_state=1,
                learning_rate=trial.suggest_float("xgb_learning_rate", 0.01, 0.3, log=True),
                n_estimators=trial.suggest_int("xgb_n_estimators", 50, 300),
                max_depth=trial.suggest_int("xgb_max_depth", 2, 10),
                min_child_weight=trial.suggest_int("xgb_min_child_weight", 1, 10),
                subsample=trial.suggest_float("xgb_subsample", 0.5, 1),
                colsample_bytree=trial.suggest_float("xgb_colsample_bytree", 0.5, 1),
                reg_alpha=trial.suggest_float("xgb_reg_alpha", 1e-3, 10.0, log=True),
                reg_lambda=trial.suggest_float("xgb_reg_lambda", 1e-3, 10.0, log=True),
#                 tree_method="gpu_hist"  # Enable GPU training
            )
            chain = RegressorChain(model, order=xgb_best_order)
            chain.fit(X_train, y_train)

        elif model_type == "lgb":
            model = LGBMRegressor(
                random_state=1,
                learning_rate=trial.suggest_float("lgb_learning_rate", 0.01, 0.3, log=True),
                n_estimators=trial.suggest_int("lgb_n_estimators", 50, 300),
                max_depth=trial.suggest_int("lgb_max_depth", 2, 10),
                num_leaves=trial.suggest_int("lgb_num_leaves", 2, 2**trial.suggest_int("lgb_max_depth", 2, 10)),
                min_child_samples=trial.suggest_int("lgb_min_child_samples", 5, 100),
                subsample=trial.suggest_float("lgb_subsample", 0.5, 1),
                colsample_bytree=trial.suggest_float("lgb_colsample_bytree", 0.5, 1),
                reg_alpha=trial.suggest_float("lgb_reg_alpha", 1e-3, 10.0, log=True),
                reg_lambda=trial.suggest_float("lgb_reg_lambda", 1e-3, 10.0, log=True),
#                 device="gpu"  # Enable GPU acceleration
            )
            chain = RegressorChain(model, order=lgb_best_order)
            chain.fit(X_train, y_train)
            
        else:
            model = CatBoostRegressor(
                random_seed=1,
                learning_rate=trial.suggest_float("cat_learning_rate", 0.01, 0.3, log=True),
                n_estimators=trial.suggest_int("cat_n_estimators", 50, 300),
                depth=trial.suggest_int("cat_depth", 2, 10),
                l2_leaf_reg=trial.suggest_float("cat_l2_leaf_reg", 1e-3, 10.0, log=True),
                border_count=trial.suggest_int("cat_border_count", 5, 200),
                subsample=trial.suggest_float("cat_subsample", 0.5, 1),
                verbose=0,
#                 task_type='GPU'  # Enable GPU training
            )
        chain = RegressorChain(model, order=cat_best_order)
        chain.fit(X_train, y_train)
        
        y_pred = chain.predict(X_dev)
        return mcrmse(y_dev, y_pred)
    
    except Exception as e:
        print(f"Error in trial {trial.number}: {e}")
        return None

pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction="minimize", pruner=pruner)
study.optimize(objective, n_trials=50)
# Actual iteration was run with n_trials = 200

best_trials = {}
for trial in study.trials:
    if trial.state == optuna.trial.TrialState.COMPLETE:
        model_type = trial.params['model_type']
        if model_type not in best_trials or trial.value < best_trials[model_type].value:
            best_trials[model_type] = trial

best_params_dict = {}
print('-' * 75)
for model_type, trial in best_trials.items():
    print(f"Best parameters for {model_type}: {trial.params}")
    print(f"Score: {trial.value}")
    print('-' * 75)
    best_params_dict[model_type] = trial.params

In [ ]:
lgb_params = best_trials['lgb'].params
xgb_params = best_trials['xgb'].params
cat_params = best_trials['cat'].params
cat_params['verbose'] = 0

# Remove the 'model_type' key from the parameters dictionaries
lgb_params.pop('model_type')
xgb_params.pop('model_type')
cat_params.pop('model_type')

# Remove model-specific prefixes from the parameter names
lgb_params = {key.replace('lgb_', ''): value for key, value in lgb_params.items()}
xgb_params = {key.replace('xgb_', ''): value for key, value in xgb_params.items()}
cat_params = {key.replace('cat_', ''): value for key, value in cat_params.items()}

# # Current best hyperparameters for the entire data, that were used for training the final model
# # XGBRegressor hyperparameters
# xgb_params = {
#     'learning_rate': 0.01257586059952504,
#     'n_estimators': 963,
#     'max_depth': 9,
#     'min_child_weight': 8,
#     'colsample_bytree': 0.4062908772431949,
#     'subsample': 0.9087746601113265,
#     'random_state': 1,
# #    'tree_method': 'gpu_hist'
# }

# # LGBMRegressor hyperparameters
# lgb_params = {
#     'learning_rate': 0.012929977832547674,
#     'n_estimators': 978,
#     'max_depth': 9,
#     'num_leaves': 232,
#     'feature_fraction': 0.38509329814179993,
#     'bagging_fraction': 0.9382159152916805,
#     'bagging_freq': 1,
#     'random_state': 1,
# #    'device': 'gpu'
# }

# # CatBoostRegressor hyperparameters
# cat_params = {
#     'learning_rate': 0.06452361591392032,
#     'iterations': 996,
#     'depth': 9,
#     'l2_leaf_reg': 8.376118948612561,
#     'verbose': 0,
#     'random_state': 1,
# #    'task_type': 'GPU'
# }

lgb_model = LGBMRegressor(**lgb_params)
xgb_model = XGBRegressor(**xgb_params)
cat_model = CatBoostRegressor(**cat_params)

## Model Training and Evaluation

In this section, we train a `RegressorChain` with each of the three models (`XGBoost`, `LightGBM`, and `CatBoost`) and the optimized order of the labels and the best params found in the previous sections. We then compute the MCRMSE on the development set for each model and use Bayesian Model Averaging (BMA) for calculating the final prediction. These weights are used to compute the ensemble predictions on the development set and the test set.

1. Define the optimized order for the `RegressorChain` and the models with the optimized hyperparameters.
2. Train a `RegressorChain` with each model and calculate the MCRMSE on the development set.
3. Calculate the BMA weights based on the inverse of the MCRMSE scores.
4. Save the trained models, MCRMSE scores, and BMA weights for later use.
5. Compute the ensemble predictions on the development set and the test set using the BMA weights and compute the MCRMSE of the ensemble predictions.

Finally, we evaluate the performance of the ensemble on the test set.



In [ ]:
%%time
# Define optimised order for the RegressorChain based on the previous code
orders = [xgb_best_order, lgb_best_order, cat_best_order] 
# Define the models based on the optimised hyperparameters
models = [xgb_model, lgb_model, cat_model]

# Train a RegressorChain with each model and calculate validation scores
chains = []
scores = []
for i, (model, order) in enumerate(zip(models, orders)):
    print(f"Training model {i+1}/{len(models)}")
    chain = RegressorChain(model, order=order, random_state=1)
    chain.fit(X_train,y_train)
    chains.append(chain)
    y_pred_dev = chain.predict(X_dev)
    score = mcrmse(y_dev, y_pred_dev)
    scores.append(score)
    print(f"Validation MCRMSE for model {i+1}: {score}")

# Calculate BMA weights based on the inverse of the validation scores
weights = [1/score for score in scores]
weights = [weight/sum(weights) for weight in weights]  # normalize so that weights sum to 1

# Save models, scores, and weights for later use
joblib.dump(chains, 'chains.joblib')
joblib.dump(scores, 'scores.joblib')
joblib.dump(weights, 'weights.joblib')

# Now, compute the ensemble predictions on the dev set and the MCRMSE
predictions_dev = [chain.predict(X_dev) for chain in chains]
final_predictions_dev = np.zeros_like(predictions_dev[0])
for weight, prediction in zip(weights, predictions_dev):
    final_predictions_dev += weight * prediction

final_score_dev = mcrmse(y_dev, final_predictions_dev)
print(f"Final ensemble validation MCRMSE: {final_score_dev}")

# Finally, evaluate on the test set
predictions_test = [chain.predict(X_test) for chain in chains]
final_predictions_test = np.zeros_like(predictions_test[0])
for weight, prediction in zip(weights, predictions_test):
    final_predictions_test += weight * prediction

final_score_test = mcrmse(y_test, final_predictions_test)
print(f"Final ensemble test MCRMSE: {final_score_test}")

## Making Predictions

Now that we have trained our model and evaluated its performance, we can use it to make predictions on new data. In this section, we will load the trained models, and BMA weights saved in the previous section and use them to compute the ensemble predictions on new data.

1. Load the Submission Feature Data and preprocess the new data in the same way as the training data.
2. Load the trained models, and BMA weights from the previous section.
3. Compute the ensemble predictions on the new data using the BMA weights.
4. Save the predictions to a file for further analysis or submission.

In [ ]:
sample = pd.read_csv("/kaggle/input/stanford-data/sample submission.csv")
sf = pd.merge(df, sample, on='DHSID', how='inner')
sf.reset_index(inplace=True, drop=True)

In [ ]:
sf.head()

In [ ]:
sf_worldbank = pd.read_csv("/kaggle/input/stanford-data/WorldBank_Sample.csv")
sf_worldbank.drop(columns = ['DHSYEAR','Country_Code','SpatialDim','TimeDim'],inplace = True)
merged_sf = pd.merge(sf, sf_worldbank, on='DHSID', how='left')

In [ ]:
merged_sf.head()

In [ ]:
merged_sf = merged_sf[selected_features]
merged_sf = one_hot_encode_transform(merged_sf, encoder)
merged_sf = pd.DataFrame(merged_sf)

In [ ]:
# Performing KNN Imputation tranformation for this data
merged_sf_numerical = merged_sf[numerical_cols].copy()
merged_sf_numerical_imputed = imputer.transform(merged_sf_numerical)

# Convert imputed data back to dataframe
merged_sf_numerical_df = pd.DataFrame(merged_sf_numerical_imputed, columns=numerical_cols, index=merged_sf.index)

# 3. Update the original sample dataset with the imputed numerical values
merged_sf.update(merged_sf_numerical_df)
merged_sf = merged_sf.fillna(X_train.median())
# merged_sf = merged_sf[X_train_cols]

In [ ]:
predictions = make_predictions(merged_sf)
predictions = pd.concat([sf['DHSID'].reset_index(),predictions],axis=1).drop(['index'],axis=1)

## Predictions
Below are the first few rows of the predictions:

In [ ]:
predictions

In [ ]:
#finally saving the final predictions as a csv
predictions.to_csv('Final_Submission.csv',index=False)